Access token: NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX

ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary


In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
def get_artist_id(artist_name, access_token):
    url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"q": artist_name}
    
    response = requests.get(url, params=params, headers=headers).json()
    hits = response["response"]["hits"]
    
    for hit in hits:
        if hit["result"]["primary_artist"]["name"].lower() == artist_name.lower():
            return hit["result"]["primary_artist"]["id"]
    return None

def get_artist_songs(artist_name, artist_id, access_token, max_songs=10):
    songs = []
    page = 1
    headers = {"Authorization": f"Bearer {access_token}"}
    
    while len(songs) < max_songs:
        url = f"https://api.genius.com/artists/{artist_id}/songs"
        params = {"per_page": 20, "page": page, "sort": "popularity"}
        response = requests.get(url, params=params, headers=headers).json()
        new_songs = response["response"]["songs"]
        
        if not new_songs:
            break
        
        for song in new_songs:
            if song["primary_artist"]["id"] == artist_id:
                songs.append({
                    "title": song["title"],
                    "url": song["url"],
                    "release_date": song.get("release_date", "N/A"),
                    "artist": artist_name
                })
                if len(songs) >= max_songs:
                    break
        page += 1
        
    return songs

def get_lyrics_from_url(song_url):
    page = requests.get(song_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    lyrics = []

    # Find the div that contains the lyrics
    lyrics_div = soup.find("div", attrs={"data-lyrics-container": "true"})
    
    # Use .decode_contents() to get the raw HTML (including <br> tags)
    lyric_links = lyrics_div.find_all("a", attrs={"data-ignore-on-click-outside": "true"})
    
    for link in lyric_links:
        # Get the raw HTML content
        lyric_html = link.find("span").decode_contents()

        # Replace <br> tags with a whitespace
        lyric_text = re.sub(r"<br\s*/?>", " ", lyric_html)
        
        # Remove bracketed sections (like [Chorus], [Verse 1], etc.)
        lyric_text = re.sub(r"\[.*?\]", "", lyric_text)
        
        # Remove parentheses (if needed)
        lyric_text = re.sub(r"[()]", "", lyric_text)
        
        lyrics.append(lyric_text)

    # Split the lyrics into words, removing punctuation, and keeping apostrophes
    words = [re.sub(r"[^\w']", '', word) for sentence in lyrics for word in sentence.split()]
    words = [word for word in words if word]  # remove empty strings

    return words
    


def get_kendrick_lyrics(access_token, max_songs=5):
    artist_id = get_artist_id("Kendrick Lamar", access_token)
    if not artist_id:
        return "Artist not found."

    songs = get_artist_songs("Kendrick Lamar", artist_id, access_token, max_songs)
    for song in songs:
        print(f"Fetching lyrics for: {song['title']}")
        lyrics = get_lyrics_from_url(song["url"])
        song["lyrics"] = lyrics
    return songs

In [3]:
# access_token = "NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX"
# kendrick_songs = get_kendrick_lyrics(access_token, max_songs=10)

# for song in kendrick_songs:
#     print(f"\n{song['title']} ({song['release_date']})\n")
#     print(song['lyrics'][:500])  # preview first 500 chars
#     print("...\n" + "-"*80)
#     print(song["url"])


In [4]:
def get_artist_lyrics(access_token, artist, max_songs=5):
    artist_id = get_artist_id(artist, access_token)
    if not artist_id:
        return "Artist not found."

    songs = get_artist_songs(artist, artist_id, access_token, max_songs)
    for song in songs:
        print(f"Fetching lyrics for: {song['title']}")
        lyrics = get_lyrics_from_url(song["url"])
        song["lyrics"] = lyrics
    return songs

In [27]:
pop_culture_artists = ["Kendrick Lamar", "Drake"]

In [28]:

song_list = []

access_token = "NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX"
all_songs = []
for artist in pop_culture_artists:
    all_songs.append(get_artist_lyrics(access_token, artist, max_songs=50))

for thing in all_songs:
    for song in thing:
        song_list.append([song["title"], song["artist"], song["lyrics"]])
    

song_info = pd.DataFrame(song_list)
song_info

Fetching lyrics for: Not Like Us
Fetching lyrics for: HUMBLE.
Fetching lyrics for: euphoria
Fetching lyrics for: m.A.A.d city
Fetching lyrics for: Swimming Pools (Drank)
Fetching lyrics for: Money Trees
Fetching lyrics for: DNA.
Fetching lyrics for: XXX.
Fetching lyrics for: meet the grahams
Fetching lyrics for: Bitch, Don’t Kill My Vibe
Fetching lyrics for: King Kunta
Fetching lyrics for: Poetic Justice
Fetching lyrics for: Alright
Fetching lyrics for: LOVE.
Fetching lyrics for: Backseat Freestyle
Fetching lyrics for: Sing About Me, I’m Dying of Thirst
Fetching lyrics for: The Blacker the Berry
Fetching lyrics for: ELEMENT.
Fetching lyrics for: LOYALTY.
Fetching lyrics for: i
Fetching lyrics for: Wesley’s Theory
Fetching lyrics for: A.D.H.D
Fetching lyrics for: FEAR.
Fetching lyrics for: All The Stars
Fetching lyrics for: PRIDE.
Fetching lyrics for: 6:16 in LA
Fetching lyrics for: Mortal Man
Fetching lyrics for: u
Fetching lyrics for: DUCKWORTH.
Fetching lyrics for: wacced out murals


,0,1,2
0,Not Like Us,Kendrick Lamar,"[Psst, I, see, dead, people, iMustard, on, the..."
1,HUMBLE.,Kendrick Lamar,"[Nobody, pray, for, me, It, been, that, day, f..."
2,euphoria,Kendrick Lamar,"[eurt, s'em, tuoba, yas, yeht, gnihtyrevE, Eup..."
3,m.A.A.d city,Kendrick Lamar,"[If, Pirus, and, Crips, all, got, along, They'..."
4,Swimming Pools (Drank),Kendrick Lamar,"[Pour, up, Drank, head, shot, Drank, Sit, down..."
...,...,...,...
95,Do Not Disturb,Drake,"[Silence, keeps, cloudin', me, Hand, on, my, h..."
96,Teenage Fever,Drake,"[Your, heart, is, hard, to, carry, after, dark..."
97,U With Me?,Drake,"[What, these, bitches, want, from, a, nigga, O..."
98,Crew Love,Drake,"[Take, your, nose, off, my, keyboard, What, yo..."


In [12]:
!pip install keras

In [29]:
song_info.columns = ["Song", "Artist", "Lyrics"]

In [30]:
from itertools import chain

def get_artist_corpus(df, artist_name):
    artist_lyrics = df[df["Artist"] == artist_name]["Lyrics"].tolist()
    return list(chain.from_iterable(artist_lyrics))

get_artist_corpus(song_info, "Drake")

['And',
 'they',
 "wishin'",
 'and',
 "wishin'",
 'And',
 "wishin'",
 'and',
 "wishin'",
 'they',
 "wishin'",
 'on',
 'me',
 'Yeah',
 'I',
 'been',
 "movin'",
 'calm',
 "don't",
 'start',
 'no',
 'trouble',
 'with',
 'me',
 'Tryna',
 'keep',
 'it',
 'peaceful',
 'is',
 'a',
 'struggle',
 'for',
 'me',
 "Don't",
 'pull',
 'up',
 'at',
 '6',
 'am',
 'to',
 'cuddle',
 'with',
 'me',
 'You',
 'know',
 'how',
 'I',
 'like',
 'it',
 'when',
 'you',
 "lovin'",
 'on',
 'me',
 'I',
 "don't",
 'wanna',
 'die',
 'for',
 'them',
 'to',
 'miss',
 'me',
 'Yes',
 'I',
 'see',
 'the',
 'things',
 'that',
 'they',
 "wishin'",
 'on',
 'me',
 'Hope',
 'I',
 'got',
 'some',
 'brothers',
 'that',
 'outlive',
 'me',
 'They',
 "gon'",
 'tell',
 'the',
 'story',
 'shit',
 'was',
 'different',
 'with',
 'me',
 "God's",
 'plan',
 "God's",
 'plan',
 'I',
 'hold',
 'back',
 'sometimes',
 'I',
 "won't",
 'yeah',
 'I',
 'feel',
 'good',
 'sometimes',
 'I',
 "don't",
 'Ayy',
 "don't",
 'I',
 'finessed',
 'down',
 'W

In [14]:
from itertools import chain
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\1khal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [16]:
def prepare_sequences(corpus, sequence_length=5):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([" ".join(corpus)])

    total_words = len(tokenizer.word_index) + 1

    # Create input sequences
    input_sequences = []
    for i in range(sequence_length, len(corpus)):
        seq = corpus[i-sequence_length:i+1]
        encoded = tokenizer.texts_to_sequences([" ".join(seq)])[0]
        input_sequences.append(encoded)

    input_sequences = np.array(input_sequences)
    X, y = input_sequences[:, :-1], input_sequences[:, -1]
    y = to_categorical(y, num_classes=total_words)

    return X, y, tokenizer, total_words

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(input_len, total_words):
    model = Sequential()
    model.add(Embedding(total_words, 64, input_length=input_len))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [31]:
corpus = get_artist_corpus(song_info, "Drake")
X, y, tokenizer, total_words = prepare_sequences(corpus)
model = build_model(X.shape[1], total_words)
model.fit(X, y, epochs=100, verbose=1)

c:\Users\1khal\miniforge3\envs\dsc80\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0341 - loss: 6.8578
Epoch 2/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.0376 - loss: 6.0644
Epoch 3/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0361 - loss: 5.9111
Epoch 4/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.0415 - loss: 5.8325
Epoch 5/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0430 - loss: 5.7352
Epoch 6/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.0462 - loss: 5.6074
Epoch 7/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.0578 - loss: 5.4616
Epoch 8/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.0694 - loss: 5.3067
Epoch 9/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.0792 - loss: 5.1377
Epoch 10/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.1025 - loss: 4.9791
Epoch 11/100
319/319 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.1227 - loss: 4.7669
Epoch 12/100
319/319 ━━━━━━━━

In [25]:
def generate_lyrics(seed_text, tokenizer, model, length=20):
    result = seed_text.split()
    for _ in range(length):
        encoded = tokenizer.texts_to_sequences([" ".join(result[-5:])])[0]
        encoded = pad_sequences([encoded], maxlen=5, truncating='pre')
        predicted = model.predict(encoded, verbose=0)
        next_word = tokenizer.index_word[np.argmax(predicted)]
        result.append(next_word)
    return " ".join(result)

In [34]:
generate_lyrics("death time baby", tokenizer, model, length=100)


"death time baby and i just love to you i feel the fuck for my name say the top is no one boy bitch i got the aampr that place i don't know that time confusin' girl that those believe mall you just fuck here uh this she started from you my face now it's just look not a whole connection this about to get so depressin' this is trauma from your own confessions this when your father leave you home alone with no protection so neglected that's why these pedophile raps and shit you so obsessed with it's so excessive they actin'"